In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.base import BaseEstimator,TransformerMixin
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.metrics import multilabel_confusion_matrix,precision_recall_curve,accuracy_score,classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))

GPU Available: True
GPU Name: Tesla T4


In [10]:
# load data
test_df = pd.read_csv(
    'test_data_solution.txt',
    sep=':::',
    engine='python',
    names=['id','title','genre','description']
)

#test data
train_df = pd.read_csv(
    'train_data.txt',
    sep=':::',
    engine='python',
    names=['id','title','genre','description']
)

In [11]:
le=LabelEncoder()


In [12]:
train_y=le.fit_transform(train_df['genre'])
test_y=le.transform(test_df['genre'])


In [13]:
test_df.isnull().sum()

,0
id,0
title,0
genre,0
description,0


In [14]:

train_df.isnull().sum()

,0
id,0
title,0
genre,0
description,0


In [15]:
test_df.genre.value_counts()

,count
genre,
drama,13612
documentary,13096
comedy,7446
short,5072
horror,2204
thriller,1590
action,1314
western,1032
reality-tv,883


In [16]:
test_df.sample(2)


,id,title,genre,description
38703,38704,A Wife's Secret (2014),romance,Frank and Bethany are happily married in a lo...
35865,35866,"""AWE Overload"" (2006)",sport,AWE Overload was a TV show from Action Wrestl...


In [17]:
test_df.title[0]


" Edgar's Lunch (1998) "

In [18]:
test_df.description[0]


" L.R. Brane loves his life - his car, his apartment, his job, but especially his girlfriend, Vespa. One day while showering, Vespa runs out of shampoo. L.R. runs across the street to a convenience store to buy some more, a quick trip of no more than a few minutes. When he returns, Vespa is gone and every trace of her existence has been wiped out. L.R.'s life becomes a tortured existence as one strange event after another occurs to confirm in his mind that a conspiracy is working against his finding Vespa."

In [19]:
test_df['tags']=test_df['title']*2+test_df['description']


In [20]:
test_df.tags[0]


" Edgar's Lunch (1998)  Edgar's Lunch (1998)  L.R. Brane loves his life - his car, his apartment, his job, but especially his girlfriend, Vespa. One day while showering, Vespa runs out of shampoo. L.R. runs across the street to a convenience store to buy some more, a quick trip of no more than a few minutes. When he returns, Vespa is gone and every trace of her existence has been wiped out. L.R.'s life becomes a tortured existence as one strange event after another occurs to confirm in his mind that a conspiracy is working against his finding Vespa."

In [21]:
test_df['tags']=test_df['tags'].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True)


In [22]:
test_df.tags[0]

' Edgars Lunch 1998  Edgars Lunch 1998  LR Brane loves his life  his car his apartment his job but especially his girlfriend Vespa One day while showering Vespa runs out of shampoo LR runs across the street to a convenience store to buy some more a quick trip of no more than a few minutes When he returns Vespa is gone and every trace of her existence has been wiped out LRs life becomes a tortured existence as one strange event after another occurs to confirm in his mind that a conspiracy is working against his finding Vespa'

In [23]:
test_df['tags']=test_df['tags'].str.lower()


In [24]:
test_df['tags'][0]


' edgars lunch 1998  edgars lunch 1998  lr brane loves his life  his car his apartment his job but especially his girlfriend vespa one day while showering vespa runs out of shampoo lr runs across the street to a convenience store to buy some more a quick trip of no more than a few minutes when he returns vespa is gone and every trace of her existence has been wiped out lrs life becomes a tortured existence as one strange event after another occurs to confirm in his mind that a conspiracy is working against his finding vespa'

In [25]:
vembed=test_df['tags'].tolist()
len(vembed)

54200

In [26]:
class data_preprocessing(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        df['tags'] = df['title']*2 + df['description']
        df['tags'] = df['tags'].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True)
        df['tags'] = df['tags'].str.lower()
        text = df['tags'].tolist()
        return text

In [27]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
class Embedder(BaseEstimator, TransformerMixin):
    def __init__(self, embeddings):
        self.embeddings = embeddings

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.embeddings.embed_documents(X)

In [29]:
x_train,x_test,y_train,y_test=train_test_split(train_df,train_y,test_size=0.3,random_state=0)


In [30]:
model=LinearSVC(class_weight='balanced')


In [31]:
pipeline1 = Pipeline([
    ("preprocessor", data_preprocessing()),
    ("embedder", Embedder(embeddings)),  # pass your embeddings model here
    ("classifier", model)
])

In [32]:
pipeline1.fit(x_train,y_train)


Pipeline(steps=[('preprocessor', data_preprocessing()),
                ('embedder',
                 Embedder(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False))),
                ('classifier', LinearSVC(class_weight='balanced'))])

In [33]:
y_p=pipeline1.predict(x_test)
print(accuracy_score(y_test,y_p))
print(classification_report(y_test, y_p))

0.41094374423608976
              precision    recall  f1-score   support

           0       0.30      0.43      0.36       376
           1       0.26      0.72      0.38       172
           2       0.14      0.20      0.17       245
           3       0.18      0.43      0.25       151
           4       0.04      0.21      0.06        80
           5       0.61      0.36      0.45      2225
           6       0.10      0.35      0.16       142
           7       0.76      0.56      0.64      3883
           8       0.72      0.28      0.40      4140
           9       0.13      0.25      0.17       248
          10       0.10      0.38      0.16       101
          11       0.37      0.71      0.49        48
          12       0.04      0.20      0.07        69
          13       0.48      0.71      0.57       658
          14       0.36      0.73      0.48       226
          15       0.08      0.42      0.14        83
          16       0.08      0.39      0.13        87
       

In [34]:
from sklearn.linear_model import LogisticRegression


In [35]:
model2=LogisticRegression(max_iter=1000)


In [36]:
pipeline2 = Pipeline([
    ("preprocessor", data_preprocessing()),
    ("embedder", Embedder(embeddings)),  # pass your embeddings model here
    ("classifier", model2)
])


In [37]:
pipeline2.fit(x_train,y_train)


Pipeline(steps=[('preprocessor', data_preprocessing()),
                ('embedder',
                 Embedder(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False))),
                ('classifier', LogisticRegression(max_iter=1000))])

In [38]:
y_p=pipeline2.predict(x_test)
print(accuracy_score(y_test,y_p))
print(classification_report(y_test, y_p))

0.5848140178296957
              precision    recall  f1-score   support

           0       0.44      0.35      0.39       376
           1       0.55      0.36      0.44       172
           2       0.33      0.11      0.17       245
           3       0.42      0.19      0.26       151
           4       0.00      0.00      0.00        80
           5       0.55      0.58      0.56      2225
           6       0.33      0.08      0.13       142
           7       0.67      0.82      0.74      3883
           8       0.57      0.74      0.64      4140
           9       0.41      0.14      0.21       248
          10       0.48      0.10      0.16       101
          11       0.83      0.60      0.70        48
          12       0.00      0.00      0.00        69
          13       0.60      0.64      0.62       658
          14       0.57      0.55      0.56       226
          15       0.55      0.07      0.13        83
          16       0.43      0.03      0.06        87
        

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
from xgboost import XGBClassifier
model3 = XGBClassifier(
    objective="multi:softmax",
    num_class=len(le.classes_),
    eval_metric="mlogloss",
    tree_method="hist",
    device="cuda",
    max_depth=6,
    learning_rate=0.1,
    n_estimators=500
)

In [40]:
pipeline3 = Pipeline([
    ("preprocessor", data_preprocessing()),
    ("embedder", Embedder(embeddings)),  # pass your embeddings model here
    ("classifier", model3)
])

In [41]:
pipeline3.fit(x_train,y_train)


Pipeline(steps=[('preprocessor', data_preprocessing()),
                ('embedder',
                 Embedder(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False))),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               c...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=500, n_jobs=None, num_class=27, ...))])

In [42]:
y_p=pipeline3.predict(x_test)
print(accuracy_score(y_test,y_p))
print(classification_report(y_test, y_p))

/usr/local/lib/python3.12/dist-packages/xgboost/core.py:751: UserWarning: [15:31:38] WARNING: /__w/xgboost/xgboost/src/common/error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


0.556470949892407
              precision    recall  f1-score   support

           0       0.49      0.21      0.30       376
           1       0.60      0.19      0.29       172
           2       0.40      0.04      0.07       245
           3       0.39      0.06      0.10       151
           4       0.00      0.00      0.00        80
           5       0.51      0.53      0.52      2225
           6       0.33      0.04      0.06       142
           7       0.63      0.84      0.72      3883
           8       0.51      0.78      0.62      4140
           9       0.51      0.09      0.15       248
          10       0.36      0.04      0.07       101
          11       0.92      0.48      0.63        48
          12       0.00      0.00      0.00        69
          13       0.65      0.51      0.57       658
          14       0.65      0.32      0.43       226
          15       0.45      0.06      0.11        83
          16       1.00      0.01      0.02        87
         

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
